In [2]:
from qiskit import QuantumCircuit
from pyscf import gto
from jqc.vqe.profile import Profile
from jqc.measure.hamiltonian import hamiltonian
from jqc.simulator.new_state_vector import StateVector

mol = gto.M(atom='H 0 0 0; H 0 0 0.74', basis='sto-3g')
profile = Profile(mol)
hamiltonian = hamiltonian(profile)

qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0, 1)

statevector = StateVector.measure(qc, hamiltonian)



(I, I, I, I)
(Z, I, I, I)
(I, I, Z, I)
(I, Z, I, I)
(I, I, I, Z)
(Z, I, Z, I)
(Z, Z, I, I)
(I, Z, Z, I)
(Z, I, I, Z)
(I, I, Z, Z)
(Y, Y, X, X)
(Y, Y, Y, Y)
(X, X, X, X)
(X, X, Y, Y)
(I, Z, I, Z)


In [5]:
import numpy as np
from scipy.sparse import csr_matrix
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from pyscf import gto
from jqc.vqe.profile import Profile
from jqc.mapper.pauli import Pauli
from jqc.measure.hamiltonian import hamiltonian
from jqc.simulator.state_vector import StateVector


# def mult_operator(statevector, operator):
#     'Multiply an operator to a statevector.'
#     indices = np.arange(len(statevector))
#     for i, op in enumerate(operator):
#         if op == Pauli.I:
#             continue
#         new_indices = indices ^ (1 << i)

#         if op == Pauli.X:
#             result = np.zeros_like(statevector)
#             result[new_indices] = statevector[indices]

#         elif op == Pauli.Y:
#             new_indices = indices ^ (1 << i)
#             result = np.zeros_like(statevector).astype(complex)
#             phase_factors = np.where((indices >> i) & 1, 1j, -1j)
#             result[new_indices] = phase_factors * statevector[indices]

#         elif op == Pauli.Z:
#             result = statevector.copy()
#             phase_factors = np.where((indices >> i) & 1, -1, 1)
#             result *= phase_factors
#     return statevector

def mult_operator(statevector, operator):
    n = len(statevector)
    indicies = np.arange(n)
    result = np.copy(statevector)

    x_mask = 0
    y_mask = 0
    z_mask = 0

    for i, op in enumerate(operator):
        if op == Pauli.X:
            x_mask |= (1 << i)
        elif op == Pauli.Y:
            y_mask |= (1 << i)
        elif op == Pauli.Z:
            z_mask |= (1 << i)

    if x_mask:
        new_indices = indicies ^ x_mask
        result[new_indices] = statevector[indicies]

    if y_mask:
        new_indices = indicies ^ y_mask
        phase_factors = np.where((indicies & y_mask) == 0, -1j, 1j)
        result[new_indices] = phase_factors * statevector[indicies]

    if z_mask:
        phase_factors = np.where((indicies & z_mask) != 0, -1, 1)
        result *= phase_factors

    return result



p_string = (Pauli.X, Pauli.Y, Pauli.Y, Pauli.X)

qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0, 1)

state_vector = Statevector.from_instruction(qc).data
state_vector2 = mult_operator(state_vector, p_string)

print(state_vector)
print(state_vector2)

# measure = sv1.conj().T @ sv2

# print(measure)


[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[0.+0.j         0.+0.j         0.+0.j         0.+0.j
 0.+0.j         0.+0.70710678j 0.-0.70710678j 0.-0.j
 0.+0.j         0.+0.j         0.+0.j         0.+0.j
 0.+0.j         0.+0.j         0.-0.j         0.-0.j        ]
